In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

In [2]:
import keras
import keras_nlp

# keras.mixed_precision.set_global_policy('mixed_bfloat16')

2024-03-20 09:29:00.159701: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 09:29:00.159833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 09:29:00.305077: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class CFG:
    seed = 42
    csv_path = "/kaggle/input/gemma-331k/gemma_331k.csv"
    preset = "gemma_instruct_2b_en"
    sequence_length = 256
    batch_size = 1
    epochs = 1
    lora_rank = 4

In [4]:
def split_data(data, num_splits: int, split_size: int):
    """
    Splits a list of data into a specified number of parts, each of a given size.

    Args:
        data: The list of data to be split.
        num_splits (int): The number of parts to divide the data into.
        split_size (int): The number of elements each part should contain.

    Returns:
        splits: array, where each inner list represents a split of the 
                original data containing `split_size` elements, except possibly 
                for the last split.

    Example usage:
        data = [1, 2, 3, ..., 50000]
        splits = split_data(data, num_splits=5, split_size=10000)
        # This will split `data` into 5 parts, each with 10000 elements.
    """
    splits = [data[i*split_size:(i+1)*split_size] for i in range(num_splits)]

    for i, split in enumerate(splits, start=1):
        print(f'Length from Split {i}: {len(split)}')

    return splits

In [5]:
import pandas as pd  

df = pd.read_csv(CFG.csv_path)
df.head()

,prep_gemma
0,Instruction:\nCreate a cover letter to apply f...
1,Instruction:\nHow would I make Python return s...
2,Instruction:\nWrite a function to design a che...
3,Instruction:\nDesign a class in Python that co...
4,Instruction:\nTurn the following tutorial for ...


In [6]:
import random

print(random.choice(df["prep_gemma"]))

Instruction:
Create a Python program that accepts periods of time from the user and calculates the total amount of time in minutes.

Response:
def calculate_total_time(text):
    total_time = 0
    # Split the text into its individual components
    components = text.split(", ")
    
    # Calculate the amount of time in minutes for each component
    for component in components:
        if "hour" in component:
            num = float(component.split(" ")[0])
            total_time += (num * 60)
        elif "minute" in component:
            num = float(component.split(" ")[0])
            total_time += num
    return total_time

total_time = calculate_total_time("3 hours, 4 minutes")
print(total_time)


In [7]:
data = df["prep_gemma"].tolist()

print(len(data) / 40)

8288.925


In [8]:
splits = split_data(data=data, num_splits=40, split_size=8288)

Length from Split 1: 8288
Length from Split 2: 8288
Length from Split 3: 8288
Length from Split 4: 8288
Length from Split 5: 8288
Length from Split 6: 8288
Length from Split 7: 8288
Length from Split 8: 8288
Length from Split 9: 8288
Length from Split 10: 8288
Length from Split 11: 8288
Length from Split 12: 8288
Length from Split 13: 8288
Length from Split 14: 8288
Length from Split 15: 8288
Length from Split 16: 8288
Length from Split 17: 8288
Length from Split 18: 8288
Length from Split 19: 8288
Length from Split 20: 8288
Length from Split 21: 8288
Length from Split 22: 8288
Length from Split 23: 8288
Length from Split 24: 8288
Length from Split 25: 8288
Length from Split 26: 8288
Length from Split 27: 8288
Length from Split 28: 8288
Length from Split 29: 8288
Length from Split 30: 8288
Length from Split 31: 8288
Length from Split 32: 8288
Length from Split 33: 8288
Length from Split 34: 8288
Length from Split 35: 8288
Length from Split 36: 8288
Length from Split 37: 8288
Length fro

In [9]:
print(f"Splits: {len(splits)}")
print(f"Values in Split 1: {len(splits[0])}")

Splits: 40
Values in Split 1: 8288


In [10]:
print(random.choice(splits[0]))

Instruction:
Create a web API with Flask and Python for requesting random articles from a database

Response:
```python
from flask import Flask, request
from flask_restful import Resource, Api
import sqlite3

app = Flask(__name__)
api = Api(app)

articles_db = 'articles.db'

class getArticle(Resource):
    def get(self):
        conn = sqlite3.connect(articles_db)
        conn.row_factory = sqlite3.Row
        c = conn.cursor()
        c.execute('SELECT * FROM articles ORDER BY RANDOM() LIMIT 1')
        data = c.fetchone()
        c.close()

        return {
            'article_id': data['article_id'],
            'title': data['title'],
            'body': data['body']
        }

api.add_resource(getArticle, '/get_article')

if __name__ == '__main__':
    app.run(debug=True)
```


In [11]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [12]:
gemma_lm.backbone.enable_lora(rank=CFG.lora_rank)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [13]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = CFG.sequence_length

# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [14]:
gemma_lm.fit(splits[0], epochs=CFG.epochs, batch_size=CFG.batch_size)

8288/8288 ━━━━━━━━━━━━━━━━━━━━ 3272s 392ms/step - loss: 0.8075 - sparse_categorical_accuracy: 0.7724


In [15]:
gemma_lm.fit(splits[1], epochs=CFG.epochs, batch_size=CFG.batch_size)

8288/8288 ━━━━━━━━━━━━━━━━━━━━ 3251s 392ms/step - loss: 0.3335 - sparse_categorical_accuracy: 0.8020


In [16]:
gemma_lm.fit(splits[2], epochs=CFG.epochs, batch_size=CFG.batch_size)

8288/8288 ━━━━━━━━━━━━━━━━━━━━ 3258s 393ms/step - loss: 0.2826 - sparse_categorical_accuracy: 0.7693


In [17]:
gemma_lm.fit(splits[3], epochs=CFG.epochs, batch_size=CFG.batch_size)

8288/8288 ━━━━━━━━━━━━━━━━━━━━ 3271s 395ms/step - loss: 1.3788 - sparse_categorical_accuracy: 0.6976


In [18]:
gemma_lm.fit(splits[4], epochs=CFG.epochs, batch_size=CFG.batch_size)

8288/8288 ━━━━━━━━━━━━━━━━━━━━ 3269s 394ms/step - loss: 1.2983 - sparse_categorical_accuracy: 0.7021


In [20]:
gemma_lm.save('gemma_41k.keras')